In [88]:
import json
import re
from pathlib import Path
import traceback




def read_file(filename):
    f = open(filename, 'r')
    return f.read()

def parse_older_format(data_str)->list:    
    line_selector = r"([0-9]{4})-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1]) (2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9]),([0-9]{3}) (.*) b'(.*)'"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        year, month, day, hour, minute, second, ms, importance, data = i
        d = json.loads(data)
        data_list.append(d)
    return data_list

def parse_newer_format(data_str)->list:    
    line_selector = r"\[(Sun|Mon|Tues|Wed|Thu|Fri|Sat), (\d{2}) (Jan|Feb|Mar|Apr|May|June|July|Aug|Sep|Oct|Nov|Dec) (\d{4}) (\d{2}):(\d{2}):(\d{2})\] INFO b'(.*)'"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        day, date, month, year, hour, minute, second, data = i
        d = json.loads(data)
        data_list.append(d)
    return data_list


def parse_ttn_log_format(data_str)->list:    
    line_selector = r"([0-9]{4})-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1]) (2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9]),([0-9]{3}) INFO  TtnListener:\d+ - Message arrived on topic 'icss_lora_tracker/devices/icspace19/up': ({.*})"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        year, month, day, hour, minute, second, ms, data = i
        d = json.loads(data)
        data_list.append(d)
    return data_list


def special_replace(raw_str,the_string):
    return raw_str.replace("{}:".format(the_string),'"{}":'.format(the_string))

def process_str(string):
    raw_str = string.replace("MSG","").replace("(","{").replace(")","}").replace("=",":").replace("'",'"')
    things_to_replace = ["app_id","dev_id","hardware_serial","counter","payload_raw","payload_fields",
                         "metadata","frequency","modulation","data_rate","airtime","coding_rate",
                        "gtw_id","gtw_trusted","channel","rssi","snr","rf_chain","latitude","longitude",
                         "location_source","analog_in_3","barometric_pressure_0","digital_out_4","gps_2",
                         "temperature_1","altitude","is_retry","antenna"]

    for i in things_to_replace:

        raw_str = raw_str.replace("{}".format(i),'"{}"'.format(i))

    raw_str = raw_str.replace("{time",'{"time"')
    raw_str = raw_str.replace("True",'true')
    raw_str = raw_str.replace("time:",'"time":')
    raw_str = raw_str.replace("fine_timestamp:",'"fine_timestamp":')
    raw_str = raw_str.replace("fine_timestamp_encrypted:",'"fine_timestamp_encrypted":')
    raw_str = raw_str.replace("timestamp:",'"timestamp":')
    raw_str = special_replace(raw_str,"port")
    raw_str = special_replace(raw_str,"gateways")

    return raw_str

def parse_jackson_log_format(data_str)->list:    
    line_selector = r"(MSG\(app_id='icss_lora_tracker', dev_id='icspace.*)"
    strings = re.findall(line_selector,data_str)
    data_list = []
    for i in strings:
        data = i
        data = process_str(data)
        if "relative_humidity_2" in data:
            continue
        if "analog_in_1" in data:
            continue

        try:
            d = json.loads(data)
        except Exception:
            print(traceback.format_exc())
            print(data)

        data_list.append(d)
    return data_list





In [89]:
filename = "mqtt_log_data-1.txt"
str_data= read_file(filename)
data_new = parse_newer_format(str_data)

In [90]:
icspace23_list = []
for i in data_new:
    dev_id = i.get('dev_id', None)
    if dev_id == "icspace23":
        icspace23_list.append(i)

In [91]:
import base64
import json
from datetime import datetime
from datetime import timezone


class PacketParser:
    def __init__(self, raw_packet):
        self.raw_packet = raw_packet
        self.current_long = 0
        self.current_lat = 0
        self.current_alt = 0
        self.current_time = 0
        self.num_sats = 0
        self.days_of_playback = 0
        self.noloadVoltage = 0
        self.loadVoltage = 0
        self.pressure = 0
        self.reset_cnt = 0
        self.boardTemp = 0
        self.device_id = ""
        self.past_positions =[]

    def parse_packet(self):
        payload_json = self.raw_packet

        date_time_str = payload_json["metadata"]["time"][:-4]
        self.current_time = datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S.%f')
        self.current_time = self.current_time.replace(tzinfo=timezone.utc)

        self.device_id = payload_json["dev_id"]


        raw_payload = payload_json["payload_raw"]

        if raw_payload == None:
            raise ValueError("No payload found")

        try:
            self.parse_raw_payload(raw_payload,self.current_time)
        except Exception as ex:
            print(ex)
        
    def parse_raw_payload(self,raw_payload,current_time):

        hex_payload = base64.b64decode(raw_payload)

        self.pressure = ((hex_payload[2] >> 1) & 0b01111111) * 10
        self.reset_cnt = hex_payload[3] & 0b00000111
        self.boardTemp = int.from_bytes(hex_payload[4:5], byteorder="little", signed=True)
        self.noloadVoltage = ((hex_payload[0] >> 3) & 0b00011111) + 18
        self.loadVoltage = (((hex_payload[0] << 2) & 0b00011100) | ((hex_payload[1] >> 6) & 0b00000011)) + 18
        self.days_of_playback = hex_payload[1] & 0b00111111
        self.num_sats = hex_payload[3] >> 3 & 0b00011111
        self.current_long = int.from_bytes(hex_payload[7:9], byteorder="little", signed=True) * 0xffff / 1e7
        self.current_lat = int.from_bytes(hex_payload[5:7], byteorder="little", signed=True) * 0xffff / 1e7
        self.current_alt = int(int.from_bytes(hex_payload[9:11], byteorder="little", signed=False) * 0xff / 1000)

        
        self.past_positions = []
        for i in range(12):
            offset = (i+1) * 9
            current_long = int.from_bytes(hex_payload[13+offset:15+offset], byteorder="little", signed=True) * 0xffff / 1e7
            current_lat = int.from_bytes(hex_payload[11+offset:13+offset], byteorder="little", signed=True) * 0xffff / 1e7
            current_alt = int(int.from_bytes(hex_payload[15+offset:17+offset], byteorder="little", signed=False) * 0xff / 1000)
            time = datetime.fromtimestamp(1577840461 + int.from_bytes(hex_payload[17+offset:20+offset], byteorder="little", signed=False) * 60)
            
            past_pos = {
                "altitude":current_alt,
                "latitude":current_lat,
                "longitude":current_long,
                "time_stamp":time

            }
            
            self.past_positions.append(past_pos)


In [92]:
for tx in icspace23_list:
    parsed_packet = PacketParser(tx)
    try:
        parsed_packet.parse_packet()
        
        payload_fields = {"barometric_pressure_0":parsed_packet.pressure,
                          "digital_out_4":parsed_packet.num_sats,
                          "gps_2":{"altitude":parsed_packet.current_alt,"latitude":parsed_packet.current_lat,"longitude":parsed_packet.current_long},
                          "temperature_1":parsed_packet.boardTemp
                          }
        print(payload_fields)
        tx["payload_fields"] = payload_fields
        
        for past_pos in parsed_packet.past_positions:
            new_pos = {
                "app_id":"icss_lora_tracker",
                "dev_id": "icspace23",
                "payload_fields":{"gps_2":{
                    "altitude":past_pos["altitude"],
                    "latitude":past_pos["latitude"],
                    "longitude":past_pos["longitude"]
                }},
                "metadata":{"time":past_pos["time_stamp"].isoformat() + 'Z'}
            }
            icspace23_list.append(new_pos)
    except:
        pass
    

{'barometric_pressure_0': 880, 'digital_out_4': 0, 'gps_2': {'altitude': 0, 'latitude': 0.0, 'longitude': 0.0}, 'temperature_1': 13}
{'barometric_pressure_0': 880, 'digital_out_4': 0, 'gps_2': {'altitude': 0, 'latitude': 0.0, 'longitude': 0.0}, 'temperature_1': 13}
{'barometric_pressure_0': 920, 'digital_out_4': 4, 'gps_2': {'altitude': 875, 'latitude': 52.008576, 'longitude': 1.5400725}, 'temperature_1': 13}
{'barometric_pressure_0': 200, 'digital_out_4': 11, 'gps_2': {'altitude': 10538, 'latitude': 51.8709525, 'longitude': 4.089384}, 'temperature_1': -29}
{'barometric_pressure_0': 200, 'digital_out_4': 11, 'gps_2': {'altitude': 10538, 'latitude': 51.8709525, 'longitude': 4.089384}, 'temperature_1': -29}
{'barometric_pressure_0': 230, 'digital_out_4': 7, 'gps_2': {'altitude': 10785, 'latitude': 51.877506, 'longitude': 4.272882}, 'temperature_1': -26}
{'barometric_pressure_0': 230, 'digital_out_4': 7, 'gps_2': {'altitude': 10785, 'latitude': 51.877506, 'longitude': 4.272882}, 'temperat

In [93]:
icspace23_list

[{'app_id': 'icss_lora_tracker',
  'dev_id': 'icspace23',
  'hardware_serial': '0093BECA9134091B',
  'port': 0,
  'counter': 42,
  'payload_raw': None,
  'metadata': {'time': '2020-12-17T12:20:54.495671829Z',
   'frequency': 868.3,
   'modulation': 'LORA',
   'data_rate': 'SF8BW125',
   'airtime': 82432000,
   'coding_rate': '4/5',
   'gateways': [{'gtw_id': 'eui-00800000a0004f45',
     'timestamp': 1144040524,
     'time': '2020-12-17T12:20:54.478213Z',
     'channel': 1,
     'rssi': -109,
     'snr': -3.2,
     'rf_chain': 0,
     'latitude': 52.25174,
     'longitude': 0.69729,
     'altitude': 70},
    {'gtw_id': 'eui-00800000a0004ead',
     'timestamp': 1675130220,
     'time': '',
     'channel': 4,
     'rssi': -116,
     'snr': -8.5,
     'rf_chain': 0},
    {'gtw_id': 'eui-00800000a0004efe',
     'timestamp': 1053695460,
     'time': '',
     'channel': 4,
     'rssi': -88,
     'snr': -7,
     'rf_chain': 0},
    {'gtw_id': 'eui-0000024b0b03045e',
     'timestamp': 283248055

In [94]:
from pymongo import MongoClient
import urllib.parse

# init mongo connection
username = urllib.parse.quote_plus('dbUser')
password = urllib.parse.quote_plus("PwBhv72bEOq4NGlI")
url = "mongodb+srv://{}:{}@cluster0.edygp.mongodb.net/test?retryWrites=true&w=majority".format(username, password)
client = MongoClient(url)

flight_data_collection = client["flight_data"]["all_flights"]

In [96]:
#flight_data_collection.insert_many(icspace23_list)